In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pylab as plt
import pandas as pd
import numpy as np
from disba import GroupDispersion
from disba import GroupSensitivity
import matplotlib
plt.rcParams['figure.figsize'] = (16,8)

# Extracting the surface wave sensitivity kernel

In this notebook, we are using a 1D velocity to extract the sensitivy kernel of the Surface Waves at the frequencies analized in previous notebooks. They provide an estimate of the sampled depth corresponding to to the variations you observed.


This can typically be done using the [Computer Program in Seismology codes (CPS)](https://www.eas.slu.edu/eqc/eqccps.html) (Herrmann, 2013). Instead we are using [disba](https://github.com/keurfonluu/disba), a computationally efficient Python library that implements a subset of codes from CPS compiled with numba.

As you will see, disba is a fast and lightweigh way to model surface wave dispersion

In [ ]:
model = pd.read_csv("PF_model.csv", usecols=[1,2,3,4])

This is a 1D velocity model extracted from [Mordret et al (2014)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2014JB011654) "Three-dimensional shear velocity anisotropic model of Piton de la Fournaise Volcano (La Réunion Island) from ambient seismic noise".
Here the velocity model coresponds to the values under the station FOR, simplified with 4 layers.


Let's formatting the array to make it compatible with disba

In [ ]:
model["depth"]=abs(model["depth"])
model["rho"]=model["rho"]
thickness = []
for i in range(len(model)-1):
    thick = abs(model.depth[i+1]-model.depth[i])
    thickness.append(thick)
thickness.append(0)

model['thickness']=thickness

In [ ]:
first = True
for i in range(len(model)-1):
    if first:
        velmod=np.array([model.thickness[i],model.vp[i],model.vs[i],model.rho[i]])
        first = False
    else:
        velmod=np.vstack((velmod,[model.thickness[i],model.vp[i],model.vs[i],model.rho[i]]))
        
#velmod=velmod[:-7]

Disba need a velocity model with thickness, Vp, Vs, density (km, km/s, km/s, g/cm3)

In [ ]:
# Velocity model
# thickness, Vp, Vs, density
# km, km/s, km/s, g/cm3
velocity_model = velmod

# Periods must be sorted starting with low periods
t = np.logspace(0.0, 1.0, 100)

# Get the 3 first Rayleigh- wave modal dispersion curves
# Fundamental mode corresponds to mode 0
gd = GroupDispersion(*velocity_model.T)
cpr = [gd(t, mode=i, wave="rayleigh") for i in range(3)]

# pd returns a namedtuple (period, velocity, mode, wave, type)

Let's take a look at the theorethical dispersion curves associated with this velocity model

In [ ]:
labels = ["Fundamental", "mode 1", "mode 2"]
plt.figure()
for i in range(3):
    plt.plot(cpr[i].period, cpr[i].velocity, label=labels[i])
    
    plt.legend()    
    plt.xlim(1,10)
plt.ylabel("velocity (km/s)")
plt.xlabel("Period(s)")
plt.title("Rayleigh group velocity dispersion curves")
plt.show()

## Extracting the sensitivity kernel

In [ ]:
gs = GroupSensitivity(*velocity_model.T)

In [ ]:
matplotlib.rcParams['axes.linewidth'] = 0.5
parameters = ["velocity_s"]
max_depth=6


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=[9,6], sharey='row',gridspec_kw={'width_ratios': [2, 2]})

# Subplot for Vp and Vs
axs[0].step(model.vp, -model.depth, label="Vp")
axs[0].step(model.vs, -model.depth, label="Vs")
axs[0].set_ylim(-max_depth,0)
#axs[0].set_xlim(3.665,3.682)
axs[0].set_xlabel('Vs (km/s)')  
axs[0].set_ylabel('Depth (km)')
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.usetex'] = False    
axs[0].tick_params(axis="y", direction='in', length=3, width=0.5)
axs[0].tick_params(axis="x", direction='in', length=3, width=0.5)
axs[0].legend(frameon=False)

periods = [0.25,0.5,2,4,6] # Choose your adventure, make list of the frequencies/periods you chose in the previous notebook

wavs = ['rayleigh']

for w, wav in enumerate(wavs):

    for p in periods: 

        sk = [gs(p, mode=0, wave=wav, parameter=parameter) for parameter in parameters]
        axs[1+w].plot(sk[0].kernel, -sk[0].depth, label = '{} s'.format(p))
        print('Period = {}'.format(p))
    axs[1+w].set_title('{}'.format(wav.capitalize()))
    axs[1+w].set_ylim(-max_depth,0)
 
    matplotlib.rcParams['text.usetex'] = True
    axs[1+w].set_xlabel('{}'.format(r'$dU/dV_{\beta}$'))
    matplotlib.rcParams['text.usetex'] = False
    axs[1+w].tick_params(axis="y", direction='in', length=3, width=0.5)
    axs[1+w].tick_params(axis="x", direction='in', length=3, width=0.5)
    axs[1+w].axvspan(-1,0, alpha=0.2,color="r")
    
axs[1].legend(frameon=False)   

axs[1].set_xlim(-0.3,1.2)
#axs[1+w].set_xlim(-0.05,0.07)
plt.tight_layout()
plt.show()

Notice that the sensitivity can be negative. Consider what that would mean for your dv/v measurements.